# Test framework of domain analysis

In [1]:
import cooler
import numpy as np
import pandas as pd
import anndata
from pathlib import Path
from glob import glob
import xarray as xr

In [2]:
from statsmodels.sandbox.stats.multicomp import multipletests as FDR
from scipy.stats import chi2_contingency

In [3]:
root = Path("/tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/Combined/hic/")

In [4]:
sc_border = anndata.read_h5ad(root / "domain" / "Subclass.boundary.h5ad")

In [5]:
sc_border

AnnData object with n_obs × n_vars = 16 × 115009
    var: 'chrom', 'start', 'end'

In [6]:
ins_object = xr.open_dataset(root / "domain" / "Subclass.insulation.nc")

In [7]:
ins_object

<xarray.Dataset>
Dimensions:                        (bin: 115009, cell: 16, type: 2)
Coordinates:
    bin_chrom                      (bin) object ...
  * bin                            (bin) int32 0 1 2 3 ... 115006 115007 115008
    bin_start                      (bin) int32 ...
    bin_end                        (bin) int32 ...
  * cell                           (cell) object 'NR2F2-LAMP5' ... 'SUB'
  * type                           (type) object 'inter' 'intra'
Data variables:
    __xarray_dataarray_variable__  (cell, bin, type) float64 ...

In [8]:
ins_scores = ins_object["__xarray_dataarray_variable__"]

In [9]:
stats = np.zeros(ins_scores.shape[2])
pv = np.ones(ins_scores.shape[2])
binfilter = (ins_scores.min(axis=(0, 1)) > 0)

In [10]:
binfilter

<xarray.DataArray '__xarray_dataarray_variable__' (type: 2)>
array([False,  True])
Coordinates:
  * type     (type) object 'inter' 'intra'

In [11]:
for i in range(ins_scores.shape[2]):
    if binfilter[i]:
        stats[i], pv[i], _, _ = chi2_contingency(ins_scores[:,:,i])

In [12]:
stats

array([  0.        , 410.54622362])

In [13]:
pv

array([1., 1.])

In [14]:
ins_scores

<xarray.DataArray '__xarray_dataarray_variable__' (cell: 16, bin: 115009,
                                                   type: 2)>
array([[[1.000000e+00, 1.000000e+00],
        [0.000000e+00, 1.964286e-01],
        ...,
        [1.312757e-03, 8.275969e-02],
        [4.917970e-04, 7.655619e-02]],

       [[1.000000e+00, 1.000000e+00],
        [0.000000e+00, 1.964286e-01],
        ...,
        [6.383218e-04, 8.084133e-02],
        [2.509767e-04, 7.440108e-02]],

       ...,

       [[1.000000e+00, 1.000000e+00],
        [0.000000e+00, 1.964286e-01],
        ...,
        [1.147695e-03, 8.038842e-02],
        [4.146663e-04, 7.376089e-02]],

       [[1.000000e+00, 1.000000e+00],
        [0.000000e+00, 1.964286e-01],
        ...,
        [1.258122e-03, 7.861334e-02],
        [4.676338e-04, 7.240772e-02]]])
Coordinates:
    bin_chrom  (bin) object ...
  * bin        (bin) int32 0 1 2 3 4 5 ... 115004 115005 115006 115007 115008
    bin_start  (bin) int32 ...
    bin_end    (bin) int32 ...
  * cell       (cell) object 'NR2F2-LAMP5' 'Micro2' 'PVALB' ... 'VIP' 'SUB'
  * type       (type) object 'inter' 'intra'

In [6]:
cool_file = cooler.Cooler(str(root / "domain_cluster_age" / "Astro" / "Age2040" / "Age2040.Q.cool"))